In [1]:
import numpy as np
import pyswarms as ps

%load_ext autoreload
%autoreload 2

In [2]:
def distance(query, target):
    x_dist = (target[0] - query[0])**2
    y_dist = (target[1] - query[1])**2
    z_dist = (target[2] - query[2])**2
    dist = np.sqrt(x_dist+y_dist+z_dist)
    return dist

[-2, 2, 3]을 위치로 inverse kinematics 풀기

In [3]:
swarm_size = 20
dim = 6        # Dimension of X
epsilon = 1.0
options = {'c1': 1.5, 'c2':1.5, 'w':0.5}

constraints = (np.array([-np.pi , -np.pi/2 , 1 , -np.pi , -5*np.pi/36 , -np.pi]),
               np.array([np.pi  ,  np.pi/2 , 3 ,  np.pi ,  5*np.pi/36 ,  np.pi]))

d1 = d2 = d3 = d4 = d5 = d6 = 3

In [4]:
def getTransformMatrix(theta, d, a, alpha):
    T = np.array([[np.cos(theta) , -np.sin(theta)*np.cos(alpha) ,  np.sin(theta)*np.sin(alpha) , a*np.cos(theta)],
                  [np.sin(theta) ,  np.cos(theta)*np.cos(alpha) , -np.cos(theta)*np.sin(alpha) , a*np.sin(theta)],
                  [0             ,  np.sin(alpha)               ,  np.cos(alpha)               , d              ],
                  [0             ,  0                           ,  0                           , 1              ]
                 ])
    return T

<img src='https://www.researchgate.net/profile/Ramabalan-Sundaresan/publication/220553251/figure/fig3/AS:585607604621312@1516631141177/Stanford-robot-manipulator.png'>

In [5]:
def get_end_tip_position(params):
    # 변환요소들을 정의
    t_00 = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
    t_01 = getTransformMatrix(params[0] , d2        , 0 , -np.pi/2)
    t_12 = getTransformMatrix(params[1] , d2        , 0 , -np.pi/2)
    t_23 = getTransformMatrix(0         , params[2] , 0 , -np.pi/2)
    t_34 = getTransformMatrix(params[3] , d4        , 0 , -np.pi/2)
    t_45 = getTransformMatrix(params[4] , 0         , 0 ,  np.pi/2)
    t_56 = getTransformMatrix(params[5] , d6        ,0  ,  0)

    # 곱해서 변환식을 정의
    end_tip_m = t_00.dot(t_01).dot(t_12).dot(t_23).dot(t_34).dot(t_45).dot(t_56)

    # The coordinates of the end tip are the 3 upper entries in the 4th column
    pos = np.array([end_tip_m[0,3],end_tip_m[1,3],end_tip_m[2,3]])
    return pos

변환 행렬도 정의  
행렬의 정의방식은 https://en.wikipedia.org/wiki/Denavit%E2%80%93Hartenberg_parameters  
theta : z축 기준 회전 각, d : 이동하는 정도, a : 회전팔, alpha : x축 기준 

변환 행렬을 정의하여 우리가 어떤 함수에 대해 최적화를 진행할지 정의해줌

In [6]:
def opt_func(X):
    n_particles = X.shape[0]  # number of particles
    target = np.array([-2,2,3]) # 각도를 찾을 점
    dist = [distance(get_end_tip_position(X[i]), target) for i in range(n_particles)]
    return np.array(dist)

변환 행렬을 이용하여 opt_func 정의

In [7]:
%%time
# Call an instance of PSO
optimizer = ps.single.GlobalBestPSO(n_particles=swarm_size,
                                    dimensions=dim,
                                    options=options,
                                    bounds=constraints)

# Perform optimization
cost, joint_vars = optimizer.optimize(opt_func, iters=1000)

2022-05-11 15:54:14,059 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.5}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|1000/1000, best_cost=0.086
2022-05-11 15:54:18,979 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0860396933019416, best pos: [-2.66357148 -0.37917355  1.10283597 -2.38611434 -0.39267426 -0.28462112]


CPU times: total: 5.16 s
Wall time: 4.92 s


In [8]:
print(get_end_tip_position(joint_vars))

[-2.06809186  2.05009895  3.01601321]


최적화를 통해 만들어낸 위치  
[-2, 2, 3]과 그렇게 차이 안남

In [9]:
cost

0.0860396933019416

In [10]:
joint_vars

array([-2.66357148, -0.37917355,  1.10283597, -2.38611434, -0.39267426,
       -0.28462112])

In [11]:
np.degrees(joint_vars)

array([-152.61140431,  -21.72504391,   63.18784668, -136.71428121,
        -22.49857808,  -16.30758878])